# Modality Prediction Consice Baseline
This notebook will levarage your expertise by our handy tools

In [1]:
from lab_scripts.data import dataloader
from lab_scripts.models.baselines import linear_regression
from lab_scripts.metrics.mp import mp_metrics
from lab_scripts.utils import utils
utils.change_directory_to_repo()

import logging
logging.basicConfig(level=logging.INFO)

In [2]:
data = dataloader.load_data('mp/official/gex_to_atac')
train_mod1 = data['train_mod1']
train_mod2 = data['train_mod2']
test_mod1 = data['test_mod1']
test_mod2 = data['test_mod2']

In [5]:
mod1 = utils.get_mod(train_mod1)
mod2 = utils.get_mod(train_mod2)
print(f'Modality of train_mod1 is {mod1}')
print(f'Modality of train_mod2 is {mod2}')

Modality of train_mod1 is gex
Modality of train_mod2 is atac


In [6]:
task_type = utils.get_task_type(mod1, mod2)
print(f'Current data type is {task_type}')

Current data type is gex_to_atac


In [7]:
svd_mod1 = linear_regression.train_svd(train_mod1.X, 50)
svd_mod2 = linear_regression.train_svd(train_mod2.X, 200)

In [8]:
train_mod1_X = svd_mod1.transform(train_mod1.X)
train_mod2_X = svd_mod2.transform(train_mod2.X)
test_mod1_X = svd_mod1.transform(test_mod1.X)

In [9]:
# Models take config as plain dicts
config = {
    'alpha': 1.0,
    'l1_ratio': 0.5,
}

In [10]:
regressor = linear_regression.train_regressor(train_mod1_X, train_mod2_X, config)

INFO:linear_regression:Fitting...


In [11]:
predictions = linear_regression.predict(test_mod1_X, regressor)

INFO:linear_regression:Predicting...


In [12]:
predictions = svd_mod2.inverse_transform(predictions)

In [13]:
mp_metrics.calculate_target(predictions, test_mod2)

0.16060328